In [1]:
import pandas as pd
import re
import sys

In [2]:
pd.set_option('display.max_columns', None)

In [77]:
abr_list = "../../annotation/resistance/Resistance_Report_snp_with_positions_v2019-01_complete.tsv"
abr_list_interest = "/home/laura/DEVELOP/SNPTB/annotation/resistance/201907/Loci_of_Interest-v5.xlsx"

In [22]:
%%bash
head -n 2 ../../annotation/resistance/Resistance_Report_snp_with_positions_v2019-01_complete.tsv

drug	gene	iso_tested	iso_w_mut	iso_excluded	denominator	nuchange	aachange	TP	FP	TN	FN	Sensitivity	Specificity	PPV	NPV	ACC	LR+	p-value	confidence_low	confidence_upp	Bonferroni_adjusted	FDR_adjusted	gene_pos	REF	ALT	Gene	is_upstream	Gene_id	start	end	strand	POS
AMIKACIN	rrs	1229	3	3	1226	1050C>T		2	1	916	307	0.0065	0.9989	0.6667	0.749	0.7488	5.93527	0.158321071658866	0.465520465996814	inf	1.0	0.803966014708151	1050	C	T	rrs		Rvnr01	1471846	1473382	+	1472895


In [49]:
df = pd.read_csv(abr_list, sep="\t")

In [50]:
df.head()

,drug,gene,iso_tested,iso_w_mut,iso_excluded,denominator,nuchange,aachange,TP,FP,TN,FN,Sensitivity,Specificity,PPV,NPV,ACC,LR+,p-value,confidence_low,confidence_upp,Bonferroni_adjusted,FDR_adjusted,gene_pos,REF,ALT,Gene,is_upstream,Gene_id,start,end,strand,POS
0,AMIKACIN,rrs,1229,3,3,1226,1050C>T,NaN,2,1,916,307,0.0065,0.9989,0.6667,0.7490,0.7488,5.93527,0.158321,0.465520,inf,1.0,0.803966,1050,C,T,rrs,NaN,Rvnr01,1471846,1473382,+,1472895
1,AMIKACIN,rrs,1229,1,3,1226,1125C>T,NaN,0,1,916,309,0.0000,0.9989,0.0000,0.7478,0.7471,0.00000,0.747961,0.000000,56.215319,1.0,0.803966,1125,C,T,rrs,NaN,Rvnr01,1471846,1473382,+,1472970
2,AMIKACIN,rrs,1229,1,3,1226,1148G>A,NaN,0,1,916,309,0.0000,0.9989,0.0000,0.7478,0.7471,0.00000,0.747961,0.000000,56.215319,1.0,0.803966,1148,G,A,rrs,NaN,Rvnr01,1471846,1473382,+,1472993
3,AMIKACIN,rrs,1229,2,3,1226,1190G>A,NaN,1,1,916,308,0.0032,0.9989,0.5000,0.7484,0.7480,2.96764,0.440708,0.077161,inf,1.0,0.803966,1190,G,A,rrs,NaN,Rvnr01,1471846,1473382,+,1473035
4,AMIKACIN,rrs,1229,1,4,1225,1223A>G,NaN,1,0,916,308,0.0032,1.0000,1.0000,0.7484,0.7486,999999.00000,0.252245,0.156151,inf,1.0,0.803966,1223,A,G,rrs,NaN,Rvnr01,1471846,1473382,+,1473068


In [51]:
df['res_gene']=df['gene'] + " (" + df['drug'] + ")"

In [64]:
df['CHROM']= "CHROM"

In [65]:
df.head()

,drug,gene,iso_tested,iso_w_mut,iso_excluded,denominator,nuchange,aachange,TP,FP,TN,FN,Sensitivity,Specificity,PPV,NPV,ACC,LR+,p-value,confidence_low,confidence_upp,Bonferroni_adjusted,FDR_adjusted,gene_pos,REF,ALT,Gene,is_upstream,Gene_id,start,end,strand,POS,res_gene,CHROM
0,AMIKACIN,rrs,1229,3,3,1226,1050C>T,NaN,2,1,916,307,0.0065,0.9989,0.6667,0.7490,0.7488,5.93527,0.158321,0.465520,inf,1.0,0.803966,1050,C,T,rrs,NaN,Rvnr01,1471846,1473382,+,1472895,rrs (AMIKACIN),CHROM
1,AMIKACIN,rrs,1229,1,3,1226,1125C>T,NaN,0,1,916,309,0.0000,0.9989,0.0000,0.7478,0.7471,0.00000,0.747961,0.000000,56.215319,1.0,0.803966,1125,C,T,rrs,NaN,Rvnr01,1471846,1473382,+,1472970,rrs (AMIKACIN),CHROM
2,AMIKACIN,rrs,1229,1,3,1226,1148G>A,NaN,0,1,916,309,0.0000,0.9989,0.0000,0.7478,0.7471,0.00000,0.747961,0.000000,56.215319,1.0,0.803966,1148,G,A,rrs,NaN,Rvnr01,1471846,1473382,+,1472993,rrs (AMIKACIN),CHROM
3,AMIKACIN,rrs,1229,2,3,1226,1190G>A,NaN,1,1,916,308,0.0032,0.9989,0.5000,0.7484,0.7480,2.96764,0.440708,0.077161,inf,1.0,0.803966,1190,G,A,rrs,NaN,Rvnr01,1471846,1473382,+,1473035,rrs (AMIKACIN),CHROM
4,AMIKACIN,rrs,1229,1,4,1225,1223A>G,NaN,1,0,916,308,0.0032,1.0000,1.0000,0.7484,0.7486,999999.00000,0.252245,0.156151,inf,1.0,0.803966,1223,A,G,rrs,NaN,Rvnr01,1471846,1473382,+,1473068,rrs (AMIKACIN),CHROM


In [67]:
df['gene'].value_counts().index.tolist()

['rpoB',
 'rpoC',
 'mas',
 'rrs',
 'katG',
 'embB',
 'pncA',
 'ppsC',
 'gyrA',
 'ppsD',
 'gidB',
 'embA',
 'gyrB',
 'ponA1',
 'rpoA',
 'embC',
 'ethA',
 'mymA',
 'eccC5',
 'mshA',
 'ndh',
 'aftA',
 'mshC',
 'ubiA',
 'clpC1',
 'kasA',
 'gpsI',
 'nudC',
 'tap',
 'embR',
 'inhA',
 'ppsA upstream',
 'rpsA',
 'Rv0565c',
 'Rv3169',
 'Rv0191',
 'Rv2731',
 'fabG1',
 'nat',
 'inbR',
 'fabG1 upstream',
 'tap upstream',
 'mshB',
 'eccB5',
 'whiB7 upstream',
 'Rv1667c',
 'sigI',
 'Rv3008',
 'Rv1667c upstream',
 'mmaA3',
 'rrl',
 'rpoB upstream',
 'eis',
 'proZ upstream',
 'ahpC',
 'ponA1 upstream',
 'furA',
 'proZ',
 'mas upstream',
 'rpsL upstream',
 'mymA upstream',
 'gyrB upstream',
 'ethR',
 'embA upstream',
 'rpsL',
 'panD',
 'ahpC upstream',
 'tlyA',
 'eis upstream',
 'whiB7',
 'mshB upstream',
 'ndh upstream',
 'gpsI upstream',
 'rpoA upstream',
 'rpsA upstream',
 'nat upstream',
 'eccB5 upstream',
 'katG upstream',
 'gyrA upstream',
 'furA upstream',
 'pncA upstream',
 'alr upstream',
 'rr

In [68]:
df_grouped = df.groupby('gene', as_index=False)

In [69]:
final_bed = df_grouped.first()[['CHROM','start','end', 'res_gene']]

In [70]:
final_bed.head()

,CHROM,start,end,res_gene
0,CHROM,222289,223530,Rv0191 (PYRAZINAMIDE)
1,CHROM,222289,223530,Rv0191 upstream (PYRAZINAMIDE)
2,CHROM,656010,657470,Rv0565c (ETHIONAMIDE)
3,CHROM,656010,657470,Rv0565c upstream (ETHIONAMIDE)
4,CHROM,1893578,1894230,Rv1667c (PYRAZINAMIDE)


In [71]:
final_bed.to_csv('../../annotation/resistance/drug_related_init.bed', sep='\t', index=False)

In [72]:
#This file will be manually modified in upstream regions and header

In [78]:
df = pd.read_excel(abr_list_interest, sep='\t', header=0)

In [80]:
df['CHROM'] = 'CHROM'

In [85]:
df['gene_res'] = df['genename'] + " (" + df['associated antibiotic resistance'] + ")"

In [86]:
df

,genename,geneid,start,stop,type,Rv number,coding strand,length,associated antibiotic resistance,Ref genome,comment,PMID Evidence,CHROM,gene_res
0,gyrB upstream,Rv0005 upstream,4998.0,5239.0,intergenic,NaN,forward,242.0,"moxifloxacin, gatifloxacin, ofloxacin, levoflo...",NaN,NaN,NaN,CHROM,"gyrB upstream (moxifloxacin, gatifloxacin, ofl..."
1,gyrB,Rv0005,5240.0,7267.0,CDS,Rv0005,forward,2028.0,"moxifloxacin, gatifloxacin, ofloxacin, levoflo...",NaN,NaN,NaN,CHROM,"gyrB (moxifloxacin, gatifloxacin, ofloxacin, l..."
2,gyrA upstream,Rv0006 upstream,7268.0,7301.0,intergenic,NaN,forward,34.0,"moxifloxacin, gatifloxacin, ofloxacin, levoflo...",NaN,NaN,NaN,CHROM,"gyrA upstream (moxifloxacin, gatifloxacin, ofl..."
3,gyrA,Rv0006,7302.0,9818.0,CDS,Rv0006,forward,2517.0,"moxifloxacin, gatifloxacin, ofloxacin, levoflo...",NaN,NaN,NaN,CHROM,"gyrA (moxifloxacin, gatifloxacin, ofloxacin, l..."
4,pbpA,Rv0016c,18759.0,20234.0,CDS,Rv0016c,reverse,1476.0,beta-lactams,NaN,NaN,NaN,CHROM,pbpA (beta-lactams)
5,ponA1 upstream,Rv0050 upstream,53245.0,53662.0,intergenic,NaN,forward,418.0,"rifampicin, rifabutin, beta-lactams",NaN,NaN,NaN,CHROM,"ponA1 upstream (rifampicin, rifabutin, beta-la..."
6,ponA1,Rv0050,53663.0,55699.0,CDS,Rv0050,forward,2037.0,"rifampicin, rifabutin, beta-lactams",NaN,NaN,NaN,CHROM,"ponA1 (rifampicin, rifabutin, beta-lactams)"
7,ldtA,Rv0116c,140267.0,141022.0,CDS,Rv0116c,reverse,756.0,beta-lactams,NaN,NaN,NaN,CHROM,ldtA (beta-lactams)
8,ldtA upstream,Rv0116c upstream,141023.0,141199.0,intergenic,NaN,reverse,177.0,beta-lactams,NaN,NaN,NaN,CHROM,ldtA upstream (beta-lactams)
9,Rv0191 upstream,Rv0191 upstream,222162.0,222288.0,intergenic,NaN,forward,127.0,pyrazinamide,NaN,NaN,NaN,CHROM,Rv0191 upstream (pyrazinamide)


In [89]:
final_interest = df[['CHROM','start','stop', 'gene_res']].dropna()

In [92]:
final_interest.start.astype(int)

0         4998
1         5240
2         7268
3         7302
4        18759
5        53245
6        53663
7       140267
8       141023
9       222162
10      222289
11      223531
12      223564
13      330933
14      331659
15      490706
16      490783
17      571649
18      571710
19      575301
20      575348
21      656010
22      657471
23      731880
24      731930
25      732393
26      732825
27      736298
28      737204
29      737268
        ...   
149    3640142
150    3640543
151    3641535
152    3840194
153    3841421
154    3877464
155    3878508
156    3986733
157    3986844
158    4007331
159    4008183
160    4038158
161    4040705
162    4043862
163    4049138
164    4049981
165    4201894
166    4202614
167    4237932
168    4239863
169    4243148
170    4243233
171    4246514
172    4268925
173    4269834
174    4326004
175    4327474
176    4327549
177    4407528
178    4408203
Name: start, Length: 179, dtype: int64

In [93]:
final_interest

,CHROM,start,stop,gene_res
0,CHROM,4998.0,5239.0,"gyrB upstream (moxifloxacin, gatifloxacin, ofl..."
1,CHROM,5240.0,7267.0,"gyrB (moxifloxacin, gatifloxacin, ofloxacin, l..."
2,CHROM,7268.0,7301.0,"gyrA upstream (moxifloxacin, gatifloxacin, ofl..."
3,CHROM,7302.0,9818.0,"gyrA (moxifloxacin, gatifloxacin, ofloxacin, l..."
4,CHROM,18759.0,20234.0,pbpA (beta-lactams)
5,CHROM,53245.0,53662.0,"ponA1 upstream (rifampicin, rifabutin, beta-la..."
6,CHROM,53663.0,55699.0,"ponA1 (rifampicin, rifabutin, beta-lactams)"
7,CHROM,140267.0,141022.0,ldtA (beta-lactams)
8,CHROM,141023.0,141199.0,ldtA upstream (beta-lactams)
9,CHROM,222162.0,222288.0,Rv0191 upstream (pyrazinamide)


In [95]:
final_interest.to_csv('../../annotation/resistance/drug_related_interest.bed', sep='\t', index=False, float_format='%.0f')